<a href="https://colab.research.google.com/github/shrinath-m-02/AgriGaurdian/blob/main/AgriGuardian_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Define the directory where your images are stored
data_dir = "/content/drive/MyDrive/AgriGuardian Data Sets /Farm Harmful Animal Dataset/Train"

# Check if the directory existsWrite a Python program that performs sentence segmentation on a paragraph containing abbreviations (e.g., “Sr.”, “U.S.A.”), decimal numbers (e.g., “3.14”), and ambiguous periods (e.g., in URLs like “x.ai”). Use regular expressions and a custom abbreviation list to avoid incorrect splits. Additionally, handle cases where sentences end with ellipses (“...”) or are part of quotations. Count the number of sentences and print each sentence with its index. Test your program on the sample paragraph below.

Sample Input:

AI is reshaping industries globally. Sr. Carlos Méndez works at x.ai. He says: “NLP is tough...” His team in U.S.A. develops models for multilingual texts. For example, their accuracy is 95.6%. However, challenges persist. Consider “bank”—is it a riverbank or a financial institution? In Beijing, 中国科学院 tackles similar issues. Their website, www.ai.cn, documents progress. Dr. Li Wei notes: “We need better algorithms.” Meanwhile, en Madrid, Srta. Ana Ruiz leads a project on Español NLP. Her team’s budget? $2.3M. They collaborate with ETH Zürich, per x.ai’s guidelines. Isn’t that cool? Ambiguities like “lead” (metal or guide) complicate things... Moreover, slang like “lol” confuses models. In 中国, Weibo posts use “666” for praise. Such nuances demand advanced processing. xAI’s Grok aims to solve this. Visit x.ai for details. Their team hopes to launch by 2026. Processing costs are high—3.14x last year’s budget. AI’s future is bright! Expected Output:

Sentence 1: AI is reshaping industries globally. Sentence 2: Sr. Carlos Méndez works at x.ai.

..

… Total Sentences: ----
if not os.path.exists(data_dir):
    print(f"Error: Directory not found: {data_dir}")
    raise FileNotFoundError(f"Directory not found: {data_dir}")

# Image dimensions - 224x224 is standard for many pre-trained models
img_height, img_width = 224, 224

# Create an enhanced ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.15,
    validation_split=0.2   # 80% training, 20% validation
)

# Load images from directories
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = train_data.num_classes
print(f"Number of classes: {num_classes}")
class_names = list(train_data.class_indices.keys())
print(f"Class names: {class_names}")

# Load MobileNetV2 - a lightweight but effective pre-trained model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model to prevent weights from being updated during training
base_model.trainable = False

# Create a model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for better training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='mobilenet_model.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

# Initial training with frozen base layers
print("Training with frozen base model...")
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Unfreeze the top layers of the base model for fine-tuning
print("Fine-tuning the model...")
base_model.trainable = True
for layer in base_model.layers[:-10]:   # Freeze all but the last 10 layers
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Final validation accuracy: {val_accuracy:.4f}")

# Save the model
model.save('mobilenet_final_model.h5')
print("Model saved successfully.")

# Make predictions on the validation data
predictions = model.predict(val_data)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = val_data.classes

# Generate the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_names)
print("\nClassification Report:")
print(report)

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show()

# Display a summary of the model architecture
model.summary()

SyntaxError: invalid character '“' (U+201C) (ipython-input-2142689396.py, line 17)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Define the directory where your images are stored
data_dir = "/content/drive/MyDrive/AgriGuardian Data Sets /Farm Harmful Animal Dataset/Train"

# Check if the directory exists
if not os.path.exists(data_dir):
    print(f"Error: Directory not found: {data_dir}")
    raise FileNotFoundError(f"Directory not found: {data_dir}")

# Image dimensions - 224x224 is standard for many pre-trained models
img_height, img_width = 224, 224

# Create an enhanced ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.15,
    validation_split=0.2   # 80% training, 20% validation
)

# Load images from directories
train_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',Write a Python program that performs sentence segmentation on a paragraph containing abbreviations (e.g., “Sr.”, “U.S.A.”), decimal numbers (e.g., “3.14”), and ambiguous periods (e.g., in URLs like “x.ai”). Use regular expressions and a custom abbreviation list to avoid incorrect splits. Additionally, handle cases where sentences end with ellipses (“...”) or are part of quotations. Count the number of sentences and print each sentence with its index. Test your program on the sample paragraph below.

Sample Input:

AI is reshaping industries globally. Sr. Carlos Méndez works at x.ai. He says: “NLP is tough...” His team in U.S.A. develops models for multilingual texts. For example, their accuracy is 95.6%. However, challenges persist. Consider “bank”—is it a riverbank or a financial institution? In Beijing, 中国科学院 tackles similar issues. Their website, www.ai.cn, documents progress. Dr. Li Wei notes: “We need better algorithms.” Meanwhile, en Madrid, Srta. Ana Ruiz leads a project on Español NLP. Her team’s budget? $2.3M. They collaborate with ETH Zürich, per x.ai’s guidelines. Isn’t that cool? Ambiguities like “lead” (metal or guide) complicate things... Moreover, slang like “lol” confuses models. In 中国, Weibo posts use “666” for praise. Such nuances demand advanced processing. xAI’s Grok aims to solve this. Visit x.ai for details. Their team hopes to launch by 2026. Processing costs are high—3.14x last year’s budget. AI’s future is bright! Expected Output:

Sentence 1: AI is reshaping industries globally. Sentence 2: Sr. Carlos Méndez works at x.ai.

..

… Total Sentences: ----
    subset='training',
    shuffle=True
)

val_data = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = train_data.num_classes
print(f"Number of classes: {num_classes}")
class_names = list(train_data.class_indices.keys())
print(f"Class names: {class_names}")

# Load MobileNetV2 - a lightweight but effective pre-trained model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)

# Freeze the base model to prevent weights from being updated during training
base_model.trainable = False

# Create a model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks for better training
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-6
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='mobilenet_model.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

# Initial training with frozen base layers
print("Training with frozen base model...")
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Unfreeze the top layers of the base model for fine-tuning
print("Fine-tuning the model...")
base_model.trainable = True
for layer in base_model.layers[:-10]:   # Freeze all but the last 10 layers
    layer.trainable = False

# Recompile with lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=callbacks
)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Final validation accuracy: {val_accuracy:.4f}")

# Save the model
model.save('mobilenet_final_model.h5')
print("Model saved successfully.")

# Make predictions on the validation data
predictions = model.predict(val_data)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = val_data.classes
Write a Python program that performs sentence segmentation on a paragraph containing abbreviations (e.g., “Sr.”, “U.S.A.”), decimal numbers (e.g., “3.14”), and ambiguous periods (e.g., in URLs like “x.ai”). Use regular expressions and a custom abbreviation list to avoid incorrect splits. Additionally, handle cases where sentences end with ellipses (“...”) or are part of quotations. Count the number of sentences and print each sentence with its index. Test your program on the sample paragraph below.

Sample Input:

AI is reshaping industries globally. Sr. Carlos Méndez works at x.ai. He says: “NLP is tough...” His team in U.S.A. develops models for multilingual texts. For example, their accuracy is 95.6%. However, challenges persist. Consider “bank”—is it a riverbank or a financial institution? In Beijing, 中国科学院 tackles similar issues. Their website, www.ai.cn, documents progress. Dr. Li Wei notes: “We need better algorithms.” Meanwhile, en Madrid, Srta. Ana Ruiz leads a project on Español NLP. Her team’s budget? $2.3M. They collaborate with ETH Zürich, per x.ai’s guidelines. Isn’t that cool? Ambiguities like “lead” (metal or guide) complicate things... Moreover, slang like “lol” confuses models. In 中国, Weibo posts use “666” for praise. Such nuances demand advanced processing. xAI’s Grok aims to solve this. Visit x.ai for details. Their team hopes to launch by 2026. Processing costs are high—3.14x last year’s budget. AI’s future is bright! Expected Output:

Sentence 1: AI is reshaping industries globally. Sentence 2: Sr. Carlos Méndez works at x.ai.

..

… Total Sentences: ----
# Generate the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_names)
print("\nClassification Report:")
print(report)

# Generate the confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Class')
plt.ylabel('True Class')
plt.title('Confusion Matrix')
plt.show()

# Display a summary of the model architecture
model.summary()

Found 73 images belonging to 6 classes.
Found 17 images belonging to 6 classes.
Number of classes: 6
Class names: ['Bear', 'Cow', 'Elephant', 'Goat', 'Rabbit', 'Wild Boar']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Training with frozen base model...


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.3571 - loss: 1.7857

3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 13s/step - accuracy: 0.3911 - loss: 1.6917 - val_accuracy: 0.9412 - val_loss: 0.1285 - learning_rate: 0.0010
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9472 - loss: 0.1414

3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.9330 - loss: 0.1602 - val_accuracy: 1.0000 - val_loss: 0.0201 - learning_rate: 0.0010
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9892 - loss: 0.0332 - val_accuracy: 1.0000 - val_loss: 0.0041 - learning_rate: 0.0010
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9892 - loss: 0.0451 - val_accuracy: 1.0000 - val_loss: 9.0578e-04 - learning_rate: 0.0010
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 1.0000 - loss: 0.0313 - val_accuracy: 1.0000 - val_loss: 0.0010 - learning_rate: 0.0010
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 1.4952e-04 - learning_rate: 0.0010
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 0.0032 - learning_rate: 0.0010
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9932 - loss: 0.0166 - val_accuracy: 1.0000 - val_loss: 1.2

Final validation accuracy: 1.0000
Model saved successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


ValueError: Number of classes, 2, does not match size of target_names, 6. Try specifying the labels parameter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('mobilenet_final_model.h5')

# Define the class labels (Ensure these match your training classes)
class_labels = ['Cow','Goat','Rabbit', 'Wild boar','Elephant','Bear']  # Update based on training data

# Image path (Replace with actual image path)
#path = '/content/drive/MyDrive/Project /test/Cow/0115.jpg'
#path = '/content/drive/MyDrive/Project /test/Goat/0058.jpg'
path = '/content/drive/MyDrive/test/Wild Boar/0012.jpg'
#path =  '/content/drive/MyDrive/Project /test/Wild Boar/0025.jpg'

# Load and preprocess the image
img = image.load_img(path, target_size=(224, 224))  # Resize image
img_array = image.img_to_array(img) / 255.0  # Convert to array and normalize
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimensionWrite a Python program that performs sentence segmentation on a paragraph containing abbreviations (e.g., “Sr.”, “U.S.A.”), decimal numbers (e.g., “3.14”), and ambiguous periods (e.g., in URLs like “x.ai”). Use regular expressions and a custom abbreviation list to avoid incorrect splits. Additionally, handle cases where sentences end with ellipses (“...”) or are part of quotations. Count the number of sentences and print each sentence with its index. Test your program on the sample paragraph below.

Sample Input:

AI is reshaping industries globally. Sr. Carlos Méndez works at x.ai. He says: “NLP is tough...” His team in U.S.A. develops models for multilingual texts. For example, their accuracy is 95.6%. However, challenges persist. Consider “bank”—is it a riverbank or a financial institution? In Beijing, 中国科学院 tackles similar issues. Their website, www.ai.cn, documents progress. Dr. Li Wei notes: “We need better algorithms.” Meanwhile, en Madrid, Srta. Ana Ruiz leads a project on Español NLP. Her team’s budget? $2.3M. They collaborate with ETH Zürich, per x.ai’s guidelines. Isn’t that cool? Ambiguities like “lead” (metal or guide) complicate things... Moreover, slang like “lol” confuses models. In 中国, Weibo posts use “666” for praise. Such nuances demand advanced processing. xAI’s Grok aims to solve this. Visit x.ai for details. Their team hopes to launch by 2026. Processing costs are high—3.14x last year’s budget. AI’s future is bright! Expected Output:

Sentence 1: AI is reshaping industries globally. Sentence 2: Sr. Carlos Méndez works at x.ai.

..

… Total Sentences: ----
# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)  # Get index of highest probability
confidence = np.max(predictions)  # Get confidence score

# Print meaningful output
predicted_label = class_labels[predicted_class]
print(f"Yes, this is a {predicted_label} picture with {confidence:.2f} confidence.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Yes, this is a Bear picture with 1.00 confidence.


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('mobilenet_final_model.h5')

# IMPORTANT: Get the actual class labels from your training data
# Instead of hardcoding, let's get them from the directory structure:
import os
data_dir = "/content/drive/MyDrive/TRAIN_dataaset"
class_labels = sorted(os.listdir(data_dir))
print(f"Model was trained on these classes: {class_labels}")

# Create the same preprocessing as during training
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Load and preprocess the image
img_path = '/content/drive/MyDrive/test/Wild Boar/0107.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Same normalization as training

# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])
confidence = float(predictions[0][predicted_class])

# Print all class probabilities for debugging
print("\nAll class probabilities:")
for i, (label, prob) in enumerate(zip(class_labels, predictions[0])):
    print(f"{label}: {prob:.4f} ({i})")

# Print result
predicted_label = class_labels[predicted_class]
print(f"\nPrediction: {predicted_label} with {confidence:.2%} confidence")

Model was trained on these classes: ['Bear', 'Cow', 'Elephant', 'Goat', 'Rabbit', 'Wild Boar']
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step

All class probabilities:
Bear: 0.0000 (0)
Cow: 0.0000 (1)
Elephant: 0.0000 (2)
Goat: 0.0000 (3)
Rabbit: 0.0000 (4)
Wild Boar: 1.0000 (5)

Prediction: Wild Boar with 100.00% confidence


**AGRISENTINAL**

In [ ]:
!pip install ultralytics roboflow opencv-python
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!mkdir -p AgriSentinel_Dataset


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive


In [ ]:
# Install ultralytics (YOLOv8), Roboflow (optional), and helpers
!pip install -q ultralytics roboflow opencv-python-headless seaborn

# show versions
import ultralytics, sys
print("Ultralytics version:", ultralytics.__version__)
import torch, torchvision
print("Torch:", torch.__version__)


Ultralytics version: 8.3.221
Torch: 2.8.0+cu126


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Adjust these paths if needed
PROJECT_ROOT = "/content/drive/MyDrive/AgriSentinel_Dataset"
DATA_YAML = f"{PROJECT_ROOT}/TRAIN_dataaset.yaml"

# Quick check
import os
print("Project root exists:", os.path.exists(PROJECT_ROOT))
print("YAML exists:", os.path.exists(DATA_YAML))
!ls -R "{PROJECT_ROOT}" | sed -n '1,200p'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Project root exists: True
YAML exists: True
/content/drive/MyDrive/AgriSentinel_Dataset:
TRAIN_dataaset
TRAIN_dataaset.yaml


In [ ]:
from ultralytics import YOLO

# Choose model (yolov8n is smallest; change to yolov8s if you have more data)
BASE_MODEL = 'yolov8n.pt'

# Create and train
model = YOLO(BASE_MODEL)

# Training config — tweak epochs / batch as needed
train_args = dict(
    data=DATA_YAML,    # path to yaml in Drive
    epochs=30,         # try 30 initially; increase if more data
    imgsz=640,         # 640 or 512 for speed
    batch=16,          # reduce if OOM
    lr=0.01,           # default lr; ultralytics auto-adjusts
    workers=4,
    patience=10,       # early stopping patience
    project=f"{PROJECT_ROOT}/yolov8_runs",
    name="exp1",
    exist_ok=True
)

# start training (this prints training progress)
model.train(**train_args)


SyntaxError: '[31m[1mlr[0m' is not a valid YOLO argument. Similar arguments are i.e. ['lrf=0.01', 'lr0=0.01'].

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-11478853-d17f-46a8-9621-e0455566d32a.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ['segment', 'obb', 'detect', 'pose', 'classify']
                MODE (required) is one of ['track', 'export', 'train', 'benchmark', 'predict', 'val']
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco8.yaml model=yolo11n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=

In [ ]:
# Show training results folder
!ls -R "{PROJECT_ROOT}/yolov8_runs/exp1" | sed -n '1,200p'

# Best model path (ultralytics saves best.pt)
best_pt = f"{PROJECT_ROOT}/yolov8_runs/exp1/weights/best.pt"
print("Best PT:", best_pt)

# Show training curves (loss, mAP)
from IPython.display import Image, display
display(Image(f"{PROJECT_ROOT}/yolov8_runs/exp1/plots/val_loss.png"))
display(Image(f"{PROJECT_ROOT}/yolov8_runs/exp1/plots/metrics.png"))


ls: cannot access '/content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1': No such file or directory
Best PT: /content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1/weights/best.pt


FileNotFoundError: No such file or directory: '/content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1/plots/val_loss.png'

FileNotFoundError: No such file or directory: '/content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1/plots/val_loss.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: '/content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1/plots/metrics.png'

FileNotFoundError: No such file or directory: '/content/drive/MyDrive/AgriSentinel_Dataset/TRAIN_dataaset/yolov8_runs/exp1/plots/metrics.png'

<IPython.core.display.Image object>